In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transformers

In [ ]:
# device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# 这种if语句的用法学习一下

In [3]:
# hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [4]:
# mnist dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True,
                                           transform=transformers.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False,
                                          transform=transformers.ToTensor())

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:03<00:00, 2542175.78it/s]


Extracting ../../data/MNIST\raw\train-images-idx3-ubyte.gz to ../../data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 159922.92it/s]


Extracting ../../data/MNIST\raw\train-labels-idx1-ubyte.gz to ../../data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:01<00:00, 1445854.33it/s]


Extracting ../../data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../../data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8280.57it/s]

Extracting ../../data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../../data/MNIST\raw



In [5]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=False)

In [6]:
# convolutional neural network
class Cnn(nn.Module):
    def __init__(self, num_classes=10):
        super(Cnn, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = Cnn(num_classes).to(device)

In [7]:
# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss:0.0834
Epoch [1/5], Step [200/600], Loss:0.0367
Epoch [1/5], Step [300/600], Loss:0.0340
Epoch [1/5], Step [400/600], Loss:0.0385
Epoch [1/5], Step [500/600], Loss:0.0134
Epoch [1/5], Step [600/600], Loss:0.0932
Epoch [2/5], Step [100/600], Loss:0.0252
Epoch [2/5], Step [200/600], Loss:0.0072
Epoch [2/5], Step [300/600], Loss:0.0349
Epoch [2/5], Step [400/600], Loss:0.0840
Epoch [2/5], Step [500/600], Loss:0.1082
Epoch [2/5], Step [600/600], Loss:0.1005
Epoch [3/5], Step [100/600], Loss:0.0278
Epoch [3/5], Step [200/600], Loss:0.1295
Epoch [3/5], Step [300/600], Loss:0.0698
Epoch [3/5], Step [400/600], Loss:0.0032
Epoch [3/5], Step [500/600], Loss:0.0664
Epoch [3/5], Step [600/600], Loss:0.0435
Epoch [4/5], Step [100/600], Loss:0.0186
Epoch [4/5], Step [200/600], Loss:0.0684
Epoch [4/5], Step [300/600], Loss:0.0040
Epoch [4/5], Step [400/600], Loss:0.0355
Epoch [4/5], Step [500/600], Loss:0.0472
Epoch [4/5], Step [600/600], Loss:0.0304
Epoch [5/5], Ste

In [9]:
# test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'
          .format(100 * correct/total))

Test Accuracy of the model on the 10000 test images: 98.97 %


In [10]:
# save the model checkpoint
torch.save(model.state_dict(), 'cnn.ckpt')